## This is a notebook that verify the communication of the pointing kernel with the Rotator
It is part of a series of notebook that tests the communication between the pointing kernel and different components:
- the mount
- the rotator
- M1M3
- M2
- Cam hexapod
- M2 hexapod
We will leave ComCam out of this notebook

The goals of the exercice are 
- Check heartbeat from all components.
- Make sure we can put all components in ENABLE state and back to STANDBY.
- Check the EFD to ensure that the correct events/commands/telemetry is published

This will ensure that we are ready to check the slew when needed


For M1M3, the only thing that needs to be done is to enable the mirror and all the rest is done internally to the M1M3. As soon as we put the mirror in enable position, the M1M3 will be already in closed-loop. The mirror is raised when it's enable. 
Needs to add the tma for this simulator. 


In [1]:
#from lsst.ts.observatory.maintel import MTCS
from lsst.ts import salobj
from lsst.ts.idl.enums import ATPtg

import asyncio

import numpy as np
from matplotlib import pyplot as plt
import astropy.units as u
from astropy.time import Time
from astropy.coordinates import AltAz, ICRS, EarthLocation, Angle, FK5
from astropy.utils import iers
iers.conf.auto_download = False

from datetime import datetime


In [ ]:
#domain = salobj.Domain()
# I don't need this if I am calling a controller using script

In [2]:
now = datetime.now()

In [3]:
start_time = datetime.now()
script = salobj.Controller("Script", index=1)
mtptg = salobj.Remote(script.domain, "MTPtg")
rot = salobj.Remote(script.domain, "Rotator")
print(f'time to start is {datetime.now() - start_time} [s]')

time to start is 0:00:07.280574 [s]


In [4]:
await asyncio.gather(mtptg.start_task,
                     script.start_task,
                     rot.start_task)

[None, None, None]

In [6]:
await mtptg.evt_heartbeat.next(flush=True, timeout=5)

In [8]:
await rot.evt_heartbeat.next(flush=True, timeout=5)

In [7]:
await mtptg.evt_summaryState.aget(timeout=5)

In [18]:
#tells me what the state of the component is
tmp= await rot.evt_summaryState.aget(timeout=5)
print(tmp)
salobj.State(tmp.summaryState)

private_revCode: a25c3281, private_sndStamp: 1592852148.7455082, private_rcvStamp: 1592883095.4188397, private_seqNum: 4, private_origin: 31, private_host: 251090770, summaryState: 2, priority: 0


<State.ENABLED: 2>

In [19]:
#This gives what is after start
help(mtptg.cmd_start)

Help on RemoteCommand in module lsst.ts.salobj.topics.remote_command object:

class RemoteCommand(lsst.ts.salobj.topics.write_topic.WriteTopic)
 |  RemoteCommand(salinfo, name)
 |  
 |  Issue a specific command topic and wait for acknowldgement.
 |  
 |  Parameters
 |  ----------
 |  salinfo : `.SalInfo`
 |      SAL component information
 |  name : `str`
 |      Command name
 |  
 |  Method resolution order:
 |      RemoteCommand
 |      lsst.ts.salobj.topics.write_topic.WriteTopic
 |      lsst.ts.salobj.topics.base_topic.BaseTopic
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, salinfo, name)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  next_ackcmd(self, ackcmd, timeout=3600, wait_done=True)
 |      Wait for the next acknowledement for the command
 |      
 |      Parameters
 |      ----------
 |      ackcmd : `SalInfo.AckCmdType`
 |          The command acknowledgement returned by
 |          the previo

In [2]:
#use this command if we want to force the start and check how bad the communication
await mtptg.cmd_start.start(timeout=5)

NameError: name 'mtptg' is not defined

In [ ]:
#if not enabled do the following: these are always safe to run

In [5]:
await salobj.set_summary_state(mtptg, salobj.State.ENABLED)

[<State.ENABLED: 2>]

In [14]:
await salobj.set_summary_state(rot, salobj.State.ENABLED)

RuntimeError: Cannot get summaryState from Rotator

In [6]:
location = EarthLocation.from_geodetic(lon=-70.747698*u.deg,
                                       lat=-30.244728*u.deg,
                                       height=2663.0*u.m)

In [ ]:
#Where do I start tracking here?

In [7]:
now = datetime.now()
print("Start to point the telescope", now)

alt = 60. * u.deg
az = 0. * u.deg
rot_tel = Angle(20, unit= u.deg) 

target_name="TMA motion test"
time_data = await mtptg.tel_timeAndDate.next(flush=True, timeout=2)
curr_time_mtptg = Time(time_data.mjd, format="mjd", scale="tai")
time_err = curr_time_mtptg - Time.now()
print(f"Time error={time_err.sec:0.2f} sec")

print(curr_time_mtptg.tai.value)

cmd_elaz = AltAz(alt=alt, az=az, 
                obstime=curr_time_mtptg.tai, 
                location=location)
cmd_radec = cmd_elaz.transform_to(ICRS)
# Calculating the other parameters     
rot_pa = rot_tel

Start to point the telescope 2020-06-23 21:19:15.801427
Time error=27.00 sec
59023.889119612686


In [8]:
#The pointing component is commanding the rot directly
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg-180,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)

print("Waiting 30s")
await asyncio.sleep(30.)

Waiting 30s


In [33]:
rot_tel = Angle(40, unit= u.deg) 
rot_pa = rot_tel

In [35]:
#it did take a while to show on chronograph so I will try sleep 1s. with rot = 60
rot_tel = Angle(60, unit= u.deg) 
rot_pa = rot_tel

In [36]:
ack = await mtptg.cmd_raDecTarget.set_start(
    targetName=target_name,
    frame=ATPtg.CoordFrame.ICRS,
    epoch=2000,  # should be ignored: no parallax or proper motion
    equinox=2000,  # should be ignored for ICRS
    ra=cmd_radec.ra.hour,
    declination=cmd_radec.dec.deg,
    parallax=0,
    pmRA=0,
    pmDec=0,
    rv=0,
    dRA=0,
    dDec=0,
    rotPA=rot_pa.deg-180,
    rotFrame=ATPtg.RotFrame.FIXED,
    rotMode=ATPtg.RotMode.FIELD,
    timeout=10
)
now = datetime.now()
print("Waiting 1s", now)
await asyncio.sleep(1.)
now = datetime.now()
print("End 1s", now)

Waiting 1s 2020-06-23 03:57:37.076113
End 1s 2020-06-23 03:57:38.077181


In [ ]:
#It still took a while... 

In [37]:
now = datetime.now()
print("when Chrono showed", now)

when Chrono showed 2020-06-23 03:58:20.550662


Check telemetry

In [25]:
inPosition = await rot.evt_inPosition.aget(timeout = 300)
print(inPosition)

private_revCode: c25b8c34, private_sndStamp: 1592884262.465725, private_rcvStamp: 1592884262.4660184, private_seqNum: 14, private_origin: 31, private_host: 251090770, inPosition: True, priority: 0


In [38]:
rotAngle = await rot.cmd_track.angle(timeout=10. )

AttributeError: 'RemoteCommand' object has no attribute 'angle'

In [39]:
help(rot.cmd_track)   
#why is there no angle? but there is one in the EFD?

Help on RemoteCommand in module lsst.ts.salobj.topics.remote_command object:

class RemoteCommand(lsst.ts.salobj.topics.write_topic.WriteTopic)
 |  RemoteCommand(salinfo, name)
 |  
 |  Issue a specific command topic and wait for acknowldgement.
 |  
 |  Parameters
 |  ----------
 |  salinfo : `.SalInfo`
 |      SAL component information
 |  name : `str`
 |      Command name
 |  
 |  Method resolution order:
 |      RemoteCommand
 |      lsst.ts.salobj.topics.write_topic.WriteTopic
 |      lsst.ts.salobj.topics.base_topic.BaseTopic
 |      abc.ABC
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, salinfo, name)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |  
 |  next_ackcmd(self, ackcmd, timeout=3600, wait_done=True)
 |      Wait for the next acknowledement for the command
 |      
 |      Parameters
 |      ----------
 |      ackcmd : `SalInfo.AckCmdType`
 |          The command acknowledgement returned by
 |          the previo

In [40]:
await mtptg.cmd_stopTracking.start(timeout=10.)

# does this put the rotator back to 0 or leave it where it is? seems to leave it where it is... is it what we want?
